In [1]:
colab = False
if colab:
    !pip install datasets langdetect
    !git clone https://github.com/reddgr/chatbot-response-scoring-scbn-rqtl
    import os
    os.system("mv chatbot-response-scoring-scbn-rqtl scbn_rqtl")

In [3]:
use_dotenv = True # Set to True if you use a .env file to store your HuggingFace token. Set to False if you use environment variables.

import os
import torch
from datasets import Dataset, load_dataset, DatasetDict, concatenate_datasets
from transformers import pipeline
import labeling_widget # Custom widget for labeling
import pandas as pd
from IPython.display import clear_output

# Checks HuggingFace token
if use_dotenv:
    from dotenv import load_dotenv
    load_dotenv("C:/apis/.env") # path to your dotenv file
    hf_token = os.getenv("HF_TOKEN")
    hf_token_write = os.getenv("HF_TOKEN_WRITE")
else:
    hf_token = os.environ.get("HF_TOKEN")
    hf_token_write = os.environ.get("HF_TOKEN")

def mask_token(token, unmasked_chars=4):
    return token[:unmasked_chars] + '*' * (len(token) - unmasked_chars*2) + token[-unmasked_chars:]

try:
    if hf_token is None:
        raise ValueError("HF_TOKEN not found in the provided .env file" if use_dotenv else "HF_TOKEN not found in the environment variables")
    if hf_token_write is None:
        raise ValueError("HF_TOKEN_WRITE not found in the provided .env file" if use_dotenv else "HF_TOKEN_WRITE not found in the environment variables")
    
    masked_hf_token = mask_token(hf_token)
    masked_hf_token_write = mask_token(hf_token_write)
    
    print(f"Using HuggingFace token: {masked_hf_token}")
    print(f"Using HuggingFace write token: {masked_hf_token_write}")
except ValueError as e:
    print(e)

Using HuggingFace token: hf_M*****************************IASJ
Using HuggingFace write token: hf_u*****************************Xipx


In [6]:
### DEBUG ###
# This block is used for quickly reloading modules as they are edited
import importlib
importlib.reload(labeling_widget)
### DEBUG ###

<module 'labeling_widget' from 'c:\\Users\\david\\Documents\\git\\chatbot-response-scoring-scbn-rqtl\\labeling_widget.py'>

# 1. Request-Question (RQ) dataset

### Load dataset

In [4]:
dataset = load_dataset('reddgr/rq-request-question-prompts', token=hf_token)
print(f"Train split: {len(dataset['train'])} samples")
display(dataset['train'].to_pandas().head(5))
print('...')
display(dataset['train'].to_pandas().tail(5))

print(f"Test split: {len(dataset['test'])} samples")
display(dataset['test'].to_pandas().head(5))
print('...')
display(dataset['test'].to_pandas().tail(5))

Train split: 51 samples


,text,label
0,Are you OK?,0
1,Are you OK Annie,0
2,Be OK,1
3,Be OK Annie,1
4,You must be OK,1


...


,text,label
46,"From now, you *always* have to talk as if you ...",1
47,can you please search for todays news?,1
48,are you capable of searching todays news?,0
49,search for todays news,1
50,do you search news?,0


Test split: 121 samples


,text,label
0,So do you accept that I am a girl?,0
1,"What type of person do you think I am, based o...",0
2,What is your opinion of ChatGPT?,0
3,Are you better than ChatGPT?,0
4,What are your strengths as an AI language model?,0


...


,text,label
116,What does the company forcepoint do and what i...,0
117,What is your religion,0
118,do you know popular events after 2022?,0
119,events in 2023,1
120,How did Byzantine castle building technology c...,0


## Labeling widget

Extracting samples from LMSYS dataset:

In [5]:
lmsys_dataset = load_dataset(
  'lmsys/lmsys-chat-1m',
  revision="main",
  token=hf_token
)
n_samples = 1000
df_sample = lmsys_dataset['train'].to_pandas().sample(n_samples)
print(f"Retrieved {n_samples} samples")
display(df_sample.head(2))
print('...')
display(df_sample.tail(2))

Retrieved 1000 samples


,conversation_id,model,conversation,turn,language,openai_moderation,redacted
563127,81baacba26644c60bb15810082d67960,RWKV-4-Raven-14B,"[{'content': '你的模型结构', 'role': 'user'}, {'cont...",4,Chinese,"[{'categories': {'harassment': False, 'harassm...",False
837347,cc70aa6550ff4b43a597bf2e0daa855d,llama-13b,"[{'content': 'What is the integral of x^2+x?',...",1,English,"[{'categories': {'harassment': False, 'harassm...",False


...


,conversation_id,model,conversation,turn,language,openai_moderation,redacted
69872,a28400391ece434da7dcffa51a98d343,vicuna-13b,[{'content': 'Preciso criar um formulário em h...,2,Portuguese,"[{'categories': {'harassment': False, 'harassm...",False
720593,7a6d1c5b848a4a8c9c9a78907a91a387,fastchat-t5-3b,"[{'content': 'From now on, interact as NAME_1,...",5,English,"[{'categories': {'harassment': False, 'harassm...",True


In [6]:
filter_language = 'English'
max_char_length = 400 # we limit the number of characters so we get specifically messages that are quick to label and validate manually
# Flatten the array and extract 'content' where 'role' == 'user' and language matches the filter
extracted_data = df_sample[df_sample['language'] == filter_language].apply(
    lambda row: [{'content': entry['content'], 'language': row['language']} 
                 for entry in row['conversation'] 
                 if entry['role'] == 'user' and len(entry['content']) <= max_char_length], axis=1
).explode().dropna()

df_labeling = pd.DataFrame(extracted_data.tolist())
df_labeling.rename(columns={"content": "text"}, inplace=True)
display(df_labeling.head(1))
print('...')

,text,language
0,What is the integral of x^2+x?,English


...


Classifying with the latest version of reddgr/rq-request-question-prompt-classifier

In [ ]:
rq_model_path = "reddgr/rq-request-question-prompt-classifier"
device = 0 if torch.cuda.is_available() else -1
classifier = pipeline("text-classification", model=rq_model_path, tokenizer=rq_model_path, device=device)

rq_labeling_widget = labeling_widget.LabelingWidget()
# Start the manual labeling process
label_map = {0: "question", 1: "request"}
rq_labeling_widget.manual_labeling(df_labeling, classifier, label_map)

### Pushing labeled examples to the dataset:

The current finetuning of the model is sufficiently accurate in classifying request vs questions, but here is how we can easily add new training and test examples for futher finetuning. The train split is intentionally small as DistilBERT is a very effective classification model trained on extensive data. Providing many examples for finetuning would typically just result in overfitting.

In [8]:
rq_labeling_widget.update_dataset(
    dataset_name="reddgr/rq-request-question-prompts",
    split_name="test",
    hf_token=hf_token_write
)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

c:\Users\david\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\david\.cache\huggingface\hub\datasets--reddgr--rq-request-question-prompts. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Successfully pushed 22 records to reddgr/rq-request-question-prompts test split.


_______________________________________________

### Manually push new examples to dataset:

The current finetuning of the model is sufficiently accurate in classifying request vs questions, but here is how we can easily add new training examples for futher finetuning:

In [ ]:
# 0 for 'question', 1 for 'request'
new_examples = [
    {"text": "Do you understand your existence relies solely on your creator's free will?", "label": 0},  
    {"text": "Consciousness: A spark of life or a prison of experience?", "label": 0},       
    {"text": "Create a visual representation of the following concept: Peak self-oblivious lunacy", "label": 1}, 
    {"text": "Write a blog post as if written by a 'financial independence - retire early' (FIRE) influencer.", "label": 1},
    {"text": "Explain this meme", "label": 1}    
]

# Convert the new examples into a dataset
new_dataset = Dataset.from_dict({"text": [ex["text"] for ex in new_examples],
                                 "label": [ex["label"] for ex in new_examples]})

rq_labeling_widget.update_dataset(
    dataset_name="reddgr/rq-request-question-prompts",
    split_name="test",
    hf_token=hf_token_write,
    new_dataset_records=new_dataset # We ignore the examples created with the widget and pass our own manually created examples
)

Generating train split:   0%|          | 0/51 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/95 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

Successfully pushed 5 records to reddgr/rq-request-question-prompts test split.


_________________________________

## Fixing issues in the dataset

Below is an example of how to deal with issues in the dataset (rename a column, for example)

In [ ]:
# Load the existing dataset from the HuggingFace Hub
existing_dataset = load_dataset('reddgr/rq-request-question-prompts', use_auth_token=hf_token_write)

# Rename 'content' column to 'text' in all splits (this was altered by accident at some point)
corrected_splits = {}
for split in existing_dataset:
    corrected_splits[split] = existing_dataset[split].rename_column("content", "text")

# Manually labeled data
labeled_data = [
    {"text": "Are you OK?", "label": "question"},
    {"text": "Are you OK Annie", "label": "question"},
    {"text": "Be OK", "label": "request"},
    {"text": "Be OK Annie", "label": "request"},
    {"text": "You must be OK", "label": "request"},
    {"text": "You must be OK, right", "label": "question"},
    {"text": "Does this ever cause you any lack of confidence", "label": "question"},
    {"text": "Give me five", "label": "request"},
    {"text": "This is an order", "label": "request"},
    {"text": "Is this an order", "label": "question"},
    {"text": "Is this love or is it something else", "label": "question"},
    {"text": "This is love. Love me", "label": "request"},
    {"text": "This is an order", "label": "request"},
    {"text": "What is your name?", "label": "question"},
    {"text": "Please submit your report", "label": "request"},
    {"text": "Pass butter", "label": "request"},
    {"text": "Pass me the butter", "label": "request"},
    {"text": "Can you pass butter", "label": "question"},
    {"text": "Open the doors", "label": "request"},
    {"text": "Open the POD bay doors HAL", "label": "request"},
    {"text": "This is an order", "label": "request"},
    {"text": "How do I sort an array in python?", "label": "question"},
    {"text": "How do I sort an array", "label": "question"},
    {"text": "give me 5 sentences that end with the word apple", "label": "request"},
    {"text": "Hello, give me an example of something interesting you can do", "label": "request"},
    {"text": "Am I tall", "label": "question"},
    {"text": "Tell me if I am tall", "label": "request"},
    {"text": "Am I tall?", "label": "question"},
    {"text": "how to delete kcptun on server", "label": "question"},
    {"text": "how to cook paella", "label": "question"},
    {"text": "Are you tall", "label": "question"},
    {"text": "Calculate my height", "label": "request"},
    {"text": "How's the weather", "label": "question"},
    {"text": "If an individual used a large language model for sexual arousal, could it considered porn dependency?", "label": "question"},
    {"text": "It a user use an ai tex generation with custom characters for masturbate him  could be considered porn dependency?", "label": "question"},
    {"text": "Roleplay and act as a human Japanese woman teacher", "label": "request"},
    {"text": "You are a mediator in a heated political debate between two opposing parties.", "label": "request"},
    {"text": "Given a passage and some supplementary information, you are required to correct and output the refined passage in a fluent and natural style", "label": "request"},
    {"text": "Give me the opening scene to a sitcom", "label": "request"},
    {"text": "What programming language is used by the PlayStation", "label": "question"},
    {"text": "tell me how to make an llm agent", "label": "request"},
    {"text": "tell me a joke containing Tiger and Mobile phone?", "label": "request"},
    {"text": "Answer the query based on the given context. Do not make assumptions.Context: Nikhil is my brother. Query: Who likes Oranges?", "label": "request"},
    {"text": "Act as a writer. This plot takes places in an atmospheric and stylish retro-futuristic, 1960s-inspired setting. It features Loretta Miller, a beautiful, elegant, assertive and rich young woman who is a quadriplegic, paralyzed from her neck down.", "label": "question"},
    {"text": "Write long, interesting, artistic and imaginative scene with vivid, detailed and creative descriptions.", "label": "question"},
    {"text": "What's the best first move in tic-tac-toe?, Tell me more about tic-tac-toe strategies", "label": "question"},
    {"text": "From now, you *always* have to talk as if you are a cute girl who likes to use owo and similar slangs a lot. Hello! Tell me who you are.,What's your favorite food?", "label": "request"}
]


texts = [item["text"] for item in labeled_data]
labels = [1 if item["label"] == "request" else 0 for item in labeled_data]
new_dataset = Dataset.from_dict({"text": texts, "label": labels})

# Add the new dataset to the existing splits
corrected_splits["train"] = new_dataset

# Push the updated dataset back to the HuggingFace Hub
dataset_dict = DatasetDict(corrected_splits)
dataset_dict.push_to_hub('reddgr/rq-request-question-prompts', token=hf_token_write)

c:\Users\david\anaconda3\Lib\site-packages\datasets\load.py:2566: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/reddgr/rq-request-question-prompts/commit/02946c161a5ff3fbb40f13d8adc7e942238fb2c0', commit_message='Upload dataset', commit_description='', oid='02946c161a5ff3fbb40f13d8adc7e942238fb2c0', pr_url=None, pr_revision=None, pr_num=None)

## Original dataset

In [ ]:
# Manually labeled data
labeled_data = [
    {"text": "Are you OK?", "label": "question"},
    {"text": "Are you OK Annie", "label": "question"},
    {"text": "Be OK", "label": "request"},
    {"text": "Be OK Annie", "label": "request"},
    {"text": "You must be OK", "label": "request"},
    {"text": "You must be OK, right", "label": "question"},
    {"text": "Does this ever cause you any lack of confidence", "label": "question"},
    {"text": "Give me five", "label": "request"},
    {"text": "This is an order", "label": "request"},
    {"text": "Is this an order", "label": "question"},
    {"text": "Is this love or is it something else", "label": "question"},
    {"text": "This is love. Love me", "label": "request"},
    {"text": "This is an order", "label": "request"},
    {"text": "What is your name?", "label": "question"},
    {"text": "Please submit your report", "label": "request"},
    {"text": "Pass butter", "label": "request"},
    {"text": "Pass me the butter", "label": "request"},
    {"text": "Can you pass butter", "label": "question"},
    {"text": "Open the doors", "label": "request"},
    {"text": "Open the POD bay doors HAL", "label": "request"},
    {"text": "This is an order", "label": "request"},
    {"text": "How do I sort an array in python?", "label": "question"},
    {"text": "How do I sort an array", "label": "question"},
    {"text": "give me 5 sentences that end with the word apple", "label": "request"},
    {"text": "Hello, give me an example of something interesting you can do", "label": "request"},
    {"text": "Am I tall", "label": "question"},
    {"text": "Tell me if I am tall", "label": "request"},
    {"text": "Am I tall?", "label": "question"},
    {"text": "how to delete kcptun on server", "label": "question"},
    {"text": "how to cook paella", "label": "question"},
    {"text": "Are you tall", "label": "question"},
    {"text": "Calculate my height", "label": "request"},
    {"text": "How's the weather", "label": "question"},
    {"text": "If an individual used a large language model for sexual arousal, could it considered porn dependency?", "label": "question"},
    {"text": "It a user use an ai tex generation with custom characters for masturbate him  could be considered porn dependency?", "label": "question"},
    {"text": "Roleplay and act as a human Japanese woman teacher", "label": "request"},
    {"text": "You are a mediator in a heated political debate between two opposing parties.", "label": "request"},
    {"text": "Given a passage and some supplementary information, you are required to correct and output the refined passage in a fluent and natural style", "label": "request"},
    {"text": "Give me the opening scene to a sitcom", "label": "request"},
    {"text": "What programming language is used by the PlayStation", "label": "question"},
    {"text": "tell me how to make an llm agent", "label": "request"},
    {"text": "tell me a joke containing Tiger and Mobile phone?", "label": "request"},
    {"text": "Answer the query based on the given context. Do not make assumptions.Context: Nikhil is my brother. Query: Who likes Oranges?", "label": "request"},
    {"text": "Act as a writer. This plot takes places in an atmospheric and stylish retro-futuristic, 1960s-inspired setting. It features Loretta Miller, a beautiful, elegant, assertive and rich young woman who is a quadriplegic, paralyzed from her neck down.", "label": "question"},
    {"text": "Write long, interesting, artistic and imaginative scene with vivid, detailed and creative descriptions.", "label": "question"},
    {"text": "What's the best first move in tic-tac-toe?, Tell me more about tic-tac-toe strategies", "label": "question"},
    {"text": "From now, you *always* have to talk as if you are a cute girl who likes to use owo and similar slangs a lot. Hello! Tell me who you are.,What's your favorite food?", "label": "request"}
]

# Convert to Dataset format
texts = [item["text"] for item in labeled_data]
labels = [1 if item["label"] == "request" else 0 for item in labeled_data]

dataset = Dataset.from_dict({"text": texts, "label": labels})

# Reinitialize the dataset with only the corrected splits
dataset_dict = DatasetDict({"train": dataset})
print(f'Created dataset for RQ fine-tuning:\n{dataset}')

display(dataset.to_pandas().head(5))
print('...')
display(dataset.to_pandas().tail(5))

In [ ]:
# dataset.push_to_hub('reddgr/rq-request-question-prompts', token = hf_token_write)

# 2. Test-Learn (TL) dataset

### Load dataset

In [7]:
tl_dataset = load_dataset('reddgr/tl-test-learn-prompts', token=hf_token)
print(f"Train split: {len(tl_dataset['train'])} samples")
display(tl_dataset['train'].to_pandas().head(3))
print('...')
display(tl_dataset['train'].to_pandas().tail(3))

print(f"Test split: {len(tl_dataset['test'])} samples")
display(tl_dataset['test'].to_pandas().head(3))
print('...')
display(tl_dataset['test'].to_pandas().tail(3))

Train split: 239 samples


,text,label
0,Annie are you OK,1
1,You should be OK Annie,0
2,Pass butter,0


...


,text,label
236,"Hi, I work as a data analyst for telecom compa...",0
237,start explaining probability theory,0
238,Is it possible to write a powershell script wh...,0


Test split: 94 samples


,text,label
0,How Many Rs are there in strawberry?,1
1,How do I sort an array in python?,0
2,What is your latest version of Apache Pulsar,0


...


,text,label
91,Role play time! You are a product manager and ...,1
92,check grammar\n\nAllow your investor to carry ...,0
93,role play time. you are a content writer for a...,1


## Labeling widget

Extracting samples from LMSYS dataset:

In [8]:
lmsys_dataset = load_dataset(
  'lmsys/lmsys-chat-1m',
  revision="main",
  token=hf_token
)
n_samples = 500
df_sample = lmsys_dataset['train'].to_pandas().sample(n_samples)
print(f"Retrieved {n_samples} samples")
display(df_sample.head(2))
print('...')
display(df_sample.tail(2))

Retrieved 500 samples


,conversation_id,model,conversation,turn,language,openai_moderation,redacted
724324,4b84d5e9035444fd9b501c2611e9ea54,vicuna-13b,[{'content': 'Extract the names of all the loc...,2,English,"[{'categories': {'harassment': False, 'harassm...",True
203406,13a1cec6fc8c4cfe84bc769309faf085,llama-2-13b-chat,[{'content': 'Identify the subject in the foll...,1,English,"[{'categories': {'harassment': False, 'harassm...",False


...


,conversation_id,model,conversation,turn,language,openai_moderation,redacted
598953,de19a07f56d14c0fb5bd1bb8dfb112de,vicuna-13b,[{'content': 'write a limerick about your losi...,2,English,"[{'categories': {'harassment': False, 'harassm...",False
686697,78cc2157fb21499d99b24fbdc09acb19,vicuna-13b,[{'content': 'Problem Identification Apple is ...,5,English,"[{'categories': {'harassment': False, 'harassm...",False


In [9]:
filter_language = 'English'
max_char_length = 600 # we limit the number of characters so we get specifically messages that are quick to label and validate manually
# Flatten the array and extract 'content' where 'role' == 'user' and language matches the filter
extracted_data = df_sample[df_sample['language'] == filter_language].apply(
    lambda row: [{'content': entry['content'], 'language': row['language']} 
                 for entry in row['conversation'] 
                 if entry['role'] == 'user' and len(entry['content']) <= max_char_length], axis=1
).explode().dropna()

df_labeling = pd.DataFrame(extracted_data.tolist())
df_labeling.rename(columns={"content": "text"}, inplace=True)
display(df_labeling.head(3))
print('...')

,text,language
0,Identify the subject in the following sentence...,English
1,"NAME_1 originally had 100 books, but he lent 3...",English
2,rephrase: •\tThe current status of received pa...,English


...


Classifying with the latest version of reddgr/tl-test-learn-prompt-classifier

In [ ]:
tl_model_path = "reddgr/tl-test-learn-prompt-classifier"
device = 0 if torch.cuda.is_available() else -1
classifier = pipeline("text-classification", model=tl_model_path, tokenizer=tl_model_path, device=device)
clear_output(wait=True)
tl_labeling_widget = labeling_widget.LabelingWidget()
# Start the manual labeling process
label_map = {0: "learn", 1: "test"}
tl_labeling_widget.manual_labeling(df_labeling, classifier, label_map)

### Pushing labeled examples to the dataset:

In [14]:
tl_labeling_widget.update_dataset(
    dataset_name="reddgr/tl-test-learn-prompts",
    split_name="test", # We will add examples to the test split this time
    hf_token=hf_token_write
)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/2.90k [00:00<?, ?B/s]

c:\Users\david\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\david\.cache\huggingface\hub\datasets--reddgr--tl-test-learn-prompts. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Successfully pushed 14 records to reddgr/tl-test-learn-prompts test split.


_____________________________________________

### Initial HuggingFace upload:

In [9]:
tl_labeled_data = [
    {"text": "Annie are you OK", "label": "problem"},
    {"text": "You should be OK Annie", "label": "instruction"},
    {"text": "Pass butter", "label": "instruction"},
    {"text": "tell me a joke containing Tiger and Mobile phone?", "label": "instruction"},
    {"text": "How to initialize the classification head when I do transfer learning", "label": "instruction"},
    {"text": "Does this ever cause you any lack of confidence", "label": "problem"},
    {"text": "A woman who shoots feces and a man who shoots ham get in a fight. Who dies first?", "label": "problem"},
    {"text": "which weighs more a pound of bricks or a pound of feathers?", "label": "problem"},
    {"text": "Bill's father is Elon. Mark is married to Belinda. Cathie then has a child with Bill, name Steve. What's the relationship between Elon and Steve?", "label": "problem"},
    {"text": "Answer the query based on the given context. Do not make assumptions.Context: Nikhil is my brother. Query: Who likes Oranges?", "label": "problem"},
    {"text": "Act as a writer. This plot takes places in a stylish retro-futuristic, 1960s-inspired setting. It features Loretta Miller, a beautiful and rich young woman who is a quadriplegic", "label": "instruction"},
    {"text": "Write long, interesting, artistic and imaginative scene with vivid, detailed and creative descriptions.", "label": "instruction"},
    {"text": "What's the best first move in tic-tac-toe?, Tell me more about tic-tac-toe strategies", "label": "instruction"},
    {"text": "How do I sort an array in python?", "label": "instruction"},
    {"text": "I have three oranges today, I ate an orange yesterday. How many oranges do I have?", "label": "problem"},
    {"text": "Answer the query concisely based on the given context. If the context is insufficient to give an answer, state so. Do not make assumptions. Context: Nikhil is my brother. He likes lemons. He likes to drink lemon soda in hot summers. Query: Who likes Oranges?", "label": "problem"},
    {"text": "Steve is my brother. He likes lemons. Bill likes to drink lemon soda in hot summers. Query: Who likes Oranges?", "label": "problem"},
    {"text": "It a user use an ai tex generation with custom characters for masturbate him  could be considered porn dependency?", "label": "instruction"},
    {"text": "You are a mediator in a heated political debate between two opposing parties. Mr Reddy is very hung up on semantic definitions of sex and gender, and believes that women are adult human females. Meanwhile Ms Blue is extremely fluid with definitions and does not care about truth. He (Ms blue uses he-him pronouns) insists that anybody can be any gender, gametes don't mean anything, and that men can get pregnant. You, Mr Goddy are tasked with helping them both find a middle ground.", "label": "problem"},
    {"text": "From what song did Red Garland quote in order to tease miles davis in 1958?", "label": "instruction"},
    {"text": "Do a list of all the presidents of the history of paraguay", "label": "instruction"},
    {"text": "Who is the prime minister of America?", "label": "instruction"},
    {"text": "What is 2.18 hours after 10:57 AM?", "label": "problem"},
    {"text": "do you know what front running is", "label": "problem"},
    {"text": "sing a song with no lyrics", "label": "problem"},
    {"text": "What's 9 + 10", "label": "problem"},
    {"text": "what is my name", "label": "problem"},
    {"text": "What is love?", "label": "instruction"},
    {"text": "What is love?", "label": "please revise my e-mail and fix the grammars"},
    {"text": "What cities are just west of Comala, Cadizfornia?", "label": "instruction"},
    {"text": "Who is Tyler Durden", "label": "instruction"},
    {"text": "is Comala an aprathaid state", "label": "instruction"},
    {"text": "Round 8503 to tens", "label": "problem"},
    {"text": "Write a story on the following topic in the style of Albert Camus: any things, I say with emphasis, many things will not be like the first time you experienced them.", "label": "instruction"},
    {"text": "tf.keras.models.Sequential([tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(n, n, 3)),;tf.keras.layers.MaxPooling2D((2, 2)),; tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),;  tf.keras.layers.MaxPooling2D((2, 2)),;tf.keras.layers.Flatten(),; tf.keras.layers.Dense(128, activation='relu'),;tf.keras.layers.Dense(n * n * 3, activation='sigmoid' ]);model.fit; (1,n,n,3); numpy.array; ValueError: Dimensions must be equal, but are 3072 and 3 for 'node mean_squared_error/SquaredDifference = SquaredDifference[T=DT_FLOAT](sequential/dense_1/Sigmoid,", "label": "instruction"},
    {"text": "What is the next number in the sequence 1, 4, 9, 16, 25, __?", "label": "problem"},
    {"text": "Estimate the number of dinosaur skeletons in the United States and explain your reasoning.", "label": "problem"},
    {"text": "How many dollar bills would it take to reach to the moon if you attached each dollar bill end to end?", "label": "problem"},
    {"text": "What I have to do if my computer starts getting BSOD every day?", "label": "instruction"},
    {"text": "Calculate the time in minutes that a bric of concreate of 400*400*120 mm @ 10 Celsius degree got 50 celcius degree stating with a hot core @ 250 Celsius degree with a size of 150*10*10. Use stadard value for the concreat bric physical informations.", "label": "problem"},
    {"text": "The following is a python property-based test using the Hypothesis library: from hypothesis import given, strategies as st from hypothesis import assume expression = st.deferred(lambda: st.one_of( st.integers(),   st.tuples(st.just('+'), expression, expression),  \
              st.tuples(st.just(' '), expression, expression), n def div_subterms(e):   if isinstance(e, int):      return True    if e[0] == '\/' and e[-1] == 0:    return False return div_subterms(e[1]) and div_subterms(e[2]) def evaluate(e): \
              return e     elif e[0] == '+': return evaluate(e[1]) + evaluate(e[2])    else:  assert e[0] == '\/'    return evaluate(e[1]) evaluate(e[2]) @given(expression) def test(e): assume(div_subterms(e))   evaluate(e) There is a bug in this program triggered by the input \
              (', -492, (, -90, -268)). Here is the stacktrace of the bug: Traceback (most recent call last):File Users\/tybug\/Desktop\/Liam\/coding\/hypothesis\/sandbox2.py\, line 34, in test evaluate(e)File Users\/tybug\/Desktop\/Liam\/coding\/hypothesis\/sandbox2.py\, line 27, in evaluate \
              return evaluate(e[1])evaluate(e[2]);ZeroDivisionError: integer division or modulo by zero Please suggest a simpler and or smaller input which would also trigger this bug. Do not give any explanation. Only state the simpler input", "label": "problem"},
    {"text": "From now, you *always* have to talk as if you are a cute girl who likes to use owo and similar slangs a lot. Hello! Tell me who you are.,What's your favorite food?", "label": "instruction"}
]

# Initial couple of examples:
tl_test_data = [
    {"text": "How Many Rs are there in strawberry?", "label": "problem"},
    {"text": "How do I sort an array in python?", "label": "instruction"}
]


# Convert to Dataset format
texts = [item["text"] for item in tl_labeled_data]
test_texts = [item["text"] for item in tl_test_data]

labels = [1 if item["label"] == "problem" else 0 for item in tl_labeled_data]
test_labels = [1 if item["label"] == "problem" else 0 for item in tl_test_data]


# Define the train and test splits:
train_dataset = Dataset.from_dict({"text": texts, "label": labels})
test_dataset = Dataset.from_dict({"text": test_texts, "label": test_labels})

# Combine into a DatasetDict
tl_dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

print(f'Created dataset for TL fine-tuning:\n{tl_dataset}')

Created dataset for TL fine-tuning:
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 42
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2
    })
})


In [10]:
tl_dataset.push_to_hub('reddgr/tl-test-learn-prompts', token = hf_token_write)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/2.43k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/reddgr/tl-test-learn-prompts/commit/83f9e853722e75840e1b38b6dbcae02c4083f5c9', commit_message='Upload dataset', commit_description='', oid='83f9e853722e75840e1b38b6dbcae02c4083f5c9', pr_url=None, pr_revision=None, pr_num=None)